compare calibration result and simulation result in the instant loop detector

In [56]:
import os
print("Current directory:", os.getcwd())
HORNSGATAN_HOME = os.environ["HORNSGATAN_HOME"]
os.chdir(HORNSGATAN_HOME)
print("Current directory:", os.getcwd())

#number = len(data)
import pandas as pd
date = '2020-01-02'
detector = 'w2e_in'
number = 10
path  = "data/sim_data/"
if number<1:
    postfix = f"{detector}_{date}"
else:
    postfix = f"{detector}_{date}_{number}"

instantInductionLoop_filename_xml = f"{path}instantInductionLoop_{postfix}.xml"


Current directory: /home/kaveh/projects/Hornsgatan
Current directory: /home/kaveh/projects/Hornsgatan


In [57]:
import xml.etree.ElementTree as ET

# Parse the XML content of output_detectors.xml
tree_output = ET.parse(instantInductionLoop_filename_xml)
root_output = tree_output.getroot()

# Extract data from the XML
instant_out_data = []
for instant_out in root_output.findall('instantOut'):
    instant_out_data.append({
        'id': instant_out.get('id'),
        'time': float(instant_out.get('time')),
        'state': instant_out.get('state'),
        'vehID': instant_out.get('vehID'),
        'speed': float(instant_out.get('speed')),  # Convert speed to km/h
        'length': instant_out.get('length'),
        'type': instant_out.get('type'),
        'gap': instant_out.get('gap')
    })

# Convert to a DataFrame
df_instant_out = pd.DataFrame(instant_out_data)
df_instant_out = df_instant_out[df_instant_out["state"]=='enter']
# Save to CSV
#output_csv_file_instant_out = 'output_instant_out.csv'
#df_instant_out.to_csv(output_csv_file_instant_out, index=False)
#print(f"Data from 'output_detectors.xml' successfully converted to CSV and saved as '{output_csv_file_instant_out}'.")


In [58]:
df_instant_out

,id,time,state,vehID,speed,length,type,gap
0,w2e_in,1.577923e+09,enter,0_w2e_in,4.58,5.00,DEFAULT_VEHTYPE,None
3,w2e_in,1.577923e+09,enter,1_w2e_in,11.17,5.00,DEFAULT_VEHTYPE,4.13
6,w2e_in,1.577923e+09,enter,2_w2e_in,8.95,5.00,DEFAULT_VEHTYPE,1.55
9,w2e_in,1.577923e+09,enter,3_w2e_in,8.20,5.00,DEFAULT_VEHTYPE,2.64
11,w2e_in,1.577923e+09,enter,4_w2e_in,9.93,5.00,DEFAULT_VEHTYPE,4.34
13,w2e_in,1.577923e+09,enter,5_w2e_in,12.59,5.00,DEFAULT_VEHTYPE,4.62
15,w2e_in,1.577923e+09,enter,6_w2e_in,14.71,5.00,DEFAULT_VEHTYPE,25.13
18,w2e_in,1.577923e+09,enter,7_w2e_in,12.24,5.00,DEFAULT_VEHTYPE,15.07
20,w2e_in,1.577923e+09,enter,8_w2e_in,12.89,5.00,DEFAULT_VEHTYPE,1.46
23,w2e_in,1.577923e+09,enter,9_w2e_in,11.63,5.00,DEFAULT_VEHTYPE,9.74


In [59]:
data = pd.read_csv(f'data/calibration_data/calibrated_data_{postfix}.csv')
data.rename(columns={"veh_id": "vehID"}, inplace=True)
data.head()

,vehID,time_detector_sim,speed_detector_sim,speed_factor,time_detector_real,depart,departSpeed,speed_detector_real,delta_time,delta_speed
0,0_w2e_in,1.577923e+09,4.998000,0.60,1577923232,1577923147,8.3340,0.0,-0.22,4.998000
1,1_w2e_in,1.577923e+09,13.328003,1.60,1577923242,1577923210,22.2240,0.0,0.08,13.328003
2,2_w2e_in,1.577923e+09,12.658974,2.15,1577923244,1577923212,29.8635,0.0,0.69,12.658974
3,3_w2e_in,1.577923e+09,15.410502,1.85,1577923247,1577923219,25.6965,0.0,0.06,15.410502
4,4_w2e_in,1.577923e+09,15.410502,1.85,1577923253,1577923225,25.6965,0.0,0.06,15.410502


In [60]:
print(len(data), len(df_instant_out))

10 10


compare time error and speed error in loop detector between simulation result and calibration result

In [61]:
compare_df = pd.merge(df_instant_out[["vehID","time","speed"]],
            data[["vehID","time_detector_sim","speed_detector_sim"
                  ,"time_detector_real","speed_detector_real"]],on="vehID")
compare_df.head()




,vehID,time,speed,time_detector_sim,speed_detector_sim,time_detector_real,speed_detector_real
0,0_w2e_in,1.577923e+09,4.58,1.577923e+09,4.998000,1577923232,0.0
1,1_w2e_in,1.577923e+09,11.17,1.577923e+09,13.328003,1577923242,0.0
2,2_w2e_in,1.577923e+09,8.95,1.577923e+09,12.658974,1577923244,0.0
3,3_w2e_in,1.577923e+09,8.20,1.577923e+09,15.410502,1577923247,0.0
4,4_w2e_in,1.577923e+09,9.93,1.577923e+09,15.410502,1577923253,0.0


In [62]:
#compare_df = compare_df.head(19)

In [63]:
time_calib_error = compare_df["time_calib_error"]=(compare_df["time_detector_sim"]-compare_df["time_detector_real"])
abs(time_calib_error).mean()

np.float64(0.15799996852874756)

In [64]:
time_sim_error = compare_df["time_sim_error"] = compare_df["time"]-compare_df["time_detector_real"]
abs(time_sim_error).mean()

np.float64(1.7199999809265136)

In [65]:
speed_calib_error= compare_df["speed_calib_error"] =compare_df["speed_detector_sim"]-compare_df["speed_detector_real"]
abs(speed_calib_error).mean()

np.float64(12.772386320681345)

In [66]:
speed_sim_error=compare_df["speed_sim_error"] =compare_df["speed"]-compare_df["speed_detector_real"]
abs(speed_sim_error).mean()

np.float64(10.689)

In [67]:
start = min(compare_df['time_sim_error'].min(),compare_df['time_calib_error'].min()) 
end = max(compare_df['time_sim_error'].max(), compare_df['time_calib_error'].max())


In [68]:
import plotly.graph_objects as go

fig = go.Figure()

xbins=dict(
        start=start,  # Bin start
        end=end,    # Bin end
        size=1    # Bin size (width of each bin)
    )
sim_MAE = round(abs(compare_df['time_sim_error']).mean(),2)
calib_MAE = round(abs(compare_df['time_calib_error']).mean(),2)

fig.add_trace(go.Histogram(x=compare_df['time_sim_error'], name=f"time_sim_error --> MAE = {sim_MAE}", opacity=0.75, xbins=xbins))
fig.add_trace(go.Histogram(x=compare_df['time_calib_error'], name=f"time_calib_error  --> MAE = {calib_MAE}", opacity=0.75, xbins=xbins))

fig.update_layout(
    barmode='group',  # Use 'overlay' if you want stacked look
    xaxis_title='time error (s)',
    yaxis_title='Count',
    title=f"error  =  simulate  -  real      ------------         {postfix}"
)

fig.show()
fig.write_html(f"diagram/hist_time_error_{postfix}.html")

In [69]:
print(compare_df['time_sim_error'].min(), compare_df['time_sim_error'].max())
print(compare_df['time_calib_error'].min(), compare_df['time_calib_error'].max())

-0.029999971389770508 6.4700000286102295
-0.2200000286102295 0.690000057220459


In [70]:
print(compare_df['speed_sim_error'].min(), compare_df['speed_sim_error'].max())
print(compare_df['speed_calib_error'].min(), compare_df['speed_calib_error'].max())

4.58 14.71
4.998 15.410501749262782


In [71]:
start = min(compare_df['speed_sim_error'].min(),compare_df['speed_calib_error'].min()) 
end = max(compare_df['speed_sim_error'].max(), compare_df['speed_calib_error'].max())

In [72]:
import plotly.graph_objects as go

fig = go.Figure()

xbins=dict(
        start=start,  # Bin start
        end=end,    # Bin end
        size=1    # Bin size (width of each bin)
    )
sim_MAE = round(abs(compare_df['speed_sim_error']).mean(),2)
calib_MAE = round(abs(compare_df['speed_calib_error']).mean(),2)
fig.add_trace(go.Histogram(x=compare_df['speed_sim_error'], name=f"speed_sim_error --> MAE = {sim_MAE}", opacity=0.75, xbins=xbins))
fig.add_trace(go.Histogram(x=compare_df['speed_calib_error'], name=f"speed_calib_error --> MAE = {calib_MAE}", opacity=0.75, xbins=xbins))

fig.update_layout(
    barmode='group',  # Use 'overlay' if you want stacked look
    xaxis_title='Value',
    yaxis_title='Count',
    title=f"error  =  simulate  -  real      ------------         {postfix}"
)

fig.show()
fig.write_html(f"diagram/hist_speed_error_{postfix}.html")

In [73]:
#import os
#os.environ["HORNSGATAN_HOME"]
